In [1]:
import bw2data as bd
import sqlite3

In [2]:
columns = {o.name for o in bd.projects.db._database.get_columns("projectdataset")}

In [10]:
ADD_FULL_HASH_COLUMN = """ALTER TABLE projectdataset ADD COLUMN "full_hash" integer default 1"""
ADD_FULL_HASH_DEFAULT = """
    BEGIN;
    CREATE TABLE "projectdataset_schema_update" (
        "id" INTEGER NOT NULL PRIMARY KEY, 
        "data" BLOB NOT NULL, 
        "name" TEXT NOT NULL, 
        "full_hash"full_hash integer default 1
    );
    INSERT INTO "projectdataset_schema_update" SELECT "id", "data", "name", "full_hash" FROM "projectdataset";
    ALTER TABLE "projectdataset" RENAME TO "projectdataset_schema_update_backup";
    ALTER TABLE "projectdataset_schema_update" RENAME TO "projectdataset";
    COMMIT;
"""

In [11]:
def update_schema(SQL):
    bd.projects.db.db.close()
    connection = sqlite3.connect(bd.projects.db._filepath)
    cursor = connection.cursor()
    cursor.executescript(SQL)
    connection.close()
    bd.projects.db._database = bd.projects.db._create_database()

In [12]:
if 'full_hash' in columns:
    update_schema(ADD_FULL_HASH_DEFAULT)
else:
    bd.projects.db._database.execute_sql(ADD_FULL_HASH_COLUMN)